## Get plain text

In [2]:
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
df = pd.read_csv("../csv/stack_overflow_transformers.csv")

In [4]:
df.head(4)

,Post Link,Title,Body
0,57248098,using huggingface's pytorch- transformers GPT-...,<p>I want to use GPT-2 to make a text classifi...
1,58454157,PyTorch BERT TypeError: forward() got an unexp...,<p>Training a BERT model using PyTorch transfo...
2,58532911,Why is the input size of the MultiheadAttentio...,<p>When using the <code>torch.nn.modules.trans...
3,59435020,Get probability of multi-token word in MASK po...,<p>It is relatively easy to get a token's prob...


In [5]:
df.shape

(489, 3)

In [6]:
content = df.iloc[0, 2]

In [6]:
def text_extraction(content):
    soup = BeautifulSoup(content, 'html.parser')
    text = ' '.join(soup.stripped_strings)
    return text

In [7]:
data = []
for i in range(df.shape[0]):
    text = df.iloc[i, 1] + text_extraction(df.iloc[i, 2])
    data.append(text)

In [8]:
data[0]

"using huggingface's pytorch- transformers GPT-2 for classifcation tasksI want to use GPT-2 to make a text classifier model. I am not really sure what head should I add after I extracted features through the GPT-2. for eample I have a sequence. import pytorch_transformers as pt \nimport torch\ntext=test.iloc[1,1]\ntext\n'If a fire wanted fanning, it could readily be fanned with a newspaper, and as the government grew weaker, I have no doubt that leather and iron acquired durability in proportion, for, in a very short time, there was not a pair of bellows in all Rotterdam that ever stood in need of a stitch or required the assistance of a hammer.'\nlen(text)\n\n74\ntokenizer = pt.GPT2Tokenizer.from_pretrained('gpt2')\nmodel = pt.GPT2Model.from_pretrained('gpt2')\nzz = tokenizer.tokenize(text)\nz1=torch.tensor([tokenizer.convert_tokens_to_ids(zz)])\nz1\ntensor([[ 1532,   257,  2046,  2227,  4336,   768,    11,   340,   714, 14704,\n           307,   277,  3577,   351,   257,  7533,    11

## Tf-idf

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [10]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(data)
print(dtm_tf.shape)

(489, 645)


In [11]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(data)
print(dtm_tfidf.shape)

(489, 645)


/home/olga/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:2029: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  warnings.warn(


## LDA

In [12]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=20, random_state=0)

## Visualization

In [23]:
!pip install pyldavis==3.4.0


In [27]:
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

In [28]:
pyLDAvis.lda_model.prepare(lda_tf, dtm_tf, tf_vectorizer)

TypeError: drop() takes from 1 to 2 positional arguments but 3 were given

In [29]:
pyLDAvis.lda_model.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

TypeError: drop() takes from 1 to 2 positional arguments but 3 were given